In [1]:
import sys

class tic_tac_toe_node:
    
    # Set of 3-index combinations that produce valid three-in-a-row
    #  We check these to see if we have any winners.  
    winners = [[0, 1 ,2], [3, 4, 5], [6, 7, 8],
               [0, 3, 6], [1, 4, 7], [2, 5, 8],
               [0, 4, 8], [2, 4, 6]]
          
    
    def __init__(self, state=None, depth=0):
        """
        Initializes a tic-tac-toe node
        """
        self.depth = depth
        self.winner = None
                
        if state == None:
            self.state = ["." for i in range(0,9)]
        else:
            self.state = state
            
    def __str__(self):
        """
        Creates a string representation of the tic-tac-toe board state
        """
        out = ""
        for i in range(0,9):
            if i%3==0: 
                out += "\n"
                
            out += self.state[i]       
            
        return out
    

    def getWinner(self):
        """
        returns the winner
        """
        self.playerCounts()
        return self.winner
            
    
    def playerCounts(self):
        """
        For each win combination, it counts the number of X's and O's in that combo.
        """
        winner = None
        counts = []
        
        # Traverse all winning cell combinations (i.e. three in a row combos)
        for win in self.winners:
            
            xCount = 0
            oCount = 0
            
            # Count number of player pieces in each location in the combo
            for i in win:
                if self.state[i] == 'X':
                    xCount += 1
                elif self.state[i] == 'O':
                    oCount+=1
            
            # Since we are counting, we can easily identify a winner and
            #  save it as a class variable.
            if xCount == 3:
                self.winner = 'X'
            if oCount == 3:
                self.winner = 'O'
            
            # Append the x and o counts for the combo.
            counts.append([xCount, oCount])
            
        return counts
    
   

    def terminal_test(self, ply):
        """
        Returns true if
        a. Max depth reached
        b. No free spaces
        c. Winner exists
        """
        
        #Depth
        if self.depth == ply:
            return True

        #Free spaces
        if self.state.count('.') == 0:
            return True
        
        # Winners?
        counts = self.playerCounts()
        if self.winner != None:
            return True
            
        return False
        
        
    
    def utility(self, player):
        """
        Determines the utility of a state relative to a given player
        """
        
        score = 0
        
        # Gets the count per row and determines if a winner
        #  exists (self.winner)
        counts = self.playerCounts()
        
        # If a player is the winner, maximum utiity
        # else, if opponent wins, the major penalty
        if self.winner != None:
            if self.winner == player:
                return 9999
            else:
                return -9999
            
        for count in counts:
        
            if (player == 'X'):
                us = count[0]
                them = count[1]
            else:
                us = count[1]
                them = count[0]
            
            if them == 0 and us > 0:
                score +=1
            
            if them > 0 and us==0:
                score -= 1
                
        return score
    
  
    def placePlayerMove(self, row, col):  
        """
        Places player's piece.  Returns true if successful.
        """
        if self.state[row*3 + col] == '.':
            self.state[row*3 + col] = 'X'
            return True
        return False
    
    def placeMove(self, index, player):
        """
        Places a player's piece at a particular index
        """
        self.state[index] = player
    
    def getSuccessors(self, player):
        """
        Creates new board states if player places a piece on one of the
        empty spaces.  This is returned as a list of successors.
        """
        empties = [i for i, val in enumerate(self.state) if val =='.']
        
        successors = []
        for i in empties:
            successor = tic_tac_toe_node(self.state[:], self.depth+1)
            successor.placeMove(i, player)
            successors.append(successor)
            
        return successors

In [8]:
class tic_tac_toe_game:
    """
    This class supports tic-tac-toe with either a minimmax (default)
    or alpha-beta pruning algorithm
    """
           
    def __init__(self, ply=2, algorithm="minimax"):
        """
        Initializes the mini-max game with a default ply
        
        algorithm=['minimax' | 'alphabeta']
        """
        self.current = tic_tac_toe_node()
        self.ply = ply
        self.algorithm = algorithm
        
    def getPlayerMove(self):
        """
        Queries the player for their next move.
        """
        valid = False
        while (valid == False):

            print("Player: Make Your Move")
            print("Row:")              
            row = int(input())
            col = int(input())
            
            # Check if input is valid (input checking and confirming open)
            if row >= 0 and row < 3 and col >=0 and col < 3:
                valid = self.current.placePlayerMove(row, col)
            
            # If move is invali
            if valid == False: 
                print("Invalid input.  Try Again.")
                
    def getAgentMove(self):
        """
        Kicks off the mini-max agent for the AI
        """
        if self.algorithm == "minimax":
            _, self.current = self.MM_Max_Value(self.current)
        else:
            _, self.current = self.AB_Max_Value(self.current, -9999, 9999)
    
    ##### Minimax Implementation of Max_Value and Min_value #####  
    def MM_Max_Value(self, state):
        """
        Implements the Max method for Mini-Max
        """
        
        if state.terminal_test(self.ply):
            return state.utility('O'), None
        
        successors = state.getSuccessors('O')
        v = -sys.maxsize-1
        best = None
        for s in successors:
            
            new_v,_ = self.MM_Min_Value(s)
            if new_v > v:
                best = s
                v = new_v
        best.depth = 0        
        return v, best
    
    
    def MM_Min_Value(self, state):
        """
        Implements the min method for Mini-Max
        """
        
        if state.terminal_test(self.ply):
            return state.utility('O'), None
        
        successors = state.getSuccessors('X')
        v = sys.maxsize
        best = None
        for s in successors:
            
            new_v,_ = self.MM_Max_Value(s)
            if new_v < v:
                best = s
                v = new_v
        best.depth = 0     
        return v, best
 
    ##### Alphabeta Pruning Implementation of Max_Value and Min_value #####
    def AB_Max_Value(self, state, alpha, beta):
        """
        Implements the Max method for Mini-Max
        """
        
        if state.terminal_test(self.ply):
            return state.utility('O'), None
        
        successors = state.getSuccessors('O')
        v = -sys.maxsize-1
        best = None
        for s in successors:
            
            #Implements v = max(v, min_value)
            new_v,_ = self.AB_Min_Value(s, alpha, beta)
            if new_v > v:
                best = s
                v = new_v
            
            # determines if pruning possible
            if v >= beta:
                print("pruning.")
                break
                
            # update alpha
            alpha = max(v, alpha)
            
        # Return the result
        best.depth = 0        
        return v, best  
    
    
    def AB_Min_Value(self, state, alpha, beta):
        """
        Implements the min method for Mini-Max
        """
        
        if state.terminal_test(self.ply):
            return state.utility('O'), None
        
        successors = state.getSuccessors('X')
        v = sys.maxsize
        best = None
        for s in successors:
            
            #Implements v = min(v, min_value)
            new_v,_ = self.AB_Max_Value(s, alpha, beta)
            if new_v < v:
                best = s
                v = new_v
            
            # determines if pruning possible
            if v <= alpha:
                print("pruning.")
                break
                
            # update beta
            beta = min(v, beta)
        best.depth = 0     
        return v, best
            
    
    def playGame(self):
        """
        Main tic-tac-toe game loop
        """
        game_over = False
        move = -1
        
        # Iteratate until the game end condition is reached
        #  Winner or stalemate
        while (self.current.terminal_test(self.ply) == False):
            move += 1
            
            if move%2 == 0:
                print("\nCurrent State:")
                print(self.current)
                self.getPlayerMove()
            else:
                self.getAgentMove()

        # Determine the winner and print the result
        winner = self.current.getWinner()
        print("\n\nGame Over:")
        if winner == None: 
            print("\tStalemate.")
        else:
            print("\t" + winner + " Wins!")
            
        print("Final State:")
        print(self.current)
                


In [9]:
# Minimax

game = tic_tac_toe_game()
game.playGame()


Current State:

...
...
...
Player: Make Your Move
Row:
1
1

Current State:

O..
.X.
...
Player: Make Your Move
Row:
2
2

Current State:

O.O
.X.
..X
Player: Make Your Move
Row:
2
0


Game Over:
	O Wins!
Final State:

OOO
.X.
X.X


In [4]:
# Alpha-Beta Pruning
game = tic_tac_toe_game(algorithm="alphabeta", ply=4)
game.playGame()


Current State:

...
...
...
Player: Make Your Move
Row:
1
1
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.
pruning.

Cu